In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_12629/3872716903.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")
/Users/kd/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
diagnostics = pd.pivot(diagnostics, 
                       columns = ['Name'], 
                       values = 'Value', 
                       index = 'FaultId')


In [4]:
# Convert columns to numeric
diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']] = diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']].apply(pd.to_numeric, errors = 'coerce')



In [ ]:
diagnostics = pd.get_dummies(diagnostics, drop_first=True, columns = ['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])
diagnostics = diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'CruiseControlActive_True', 'IgnStatus_True', 'ParkingBrake_True']]
diagnostics_orig = diagnostics.copy()

In [22]:
diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 1 to 1248458
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AcceleratorPedal           531880 non-null   float64
 1   BarometricPressure         585952 non-null   float64
 2   CruiseControlActive        574916 non-null   object 
 3   CruiseControlSetSpeed      576458 non-null   float64
 4   DistanceLtd                585798 non-null   float64
 5   EngineCoolantTemperature   586054 non-null   float64
 6   EngineLoad                 585621 non-null   float64
 7   EngineOilPressure          586220 non-null   float64
 8   EngineOilTemperature       583890 non-null   float64
 9   EngineRpm                  586909 non-null   float64
 10  EngineTimeLtd              581343 non-null   float64
 11  FuelLevel                  502777 non-null   float64
 12  FuelLtd                    585173 non-null   float64
 13  FuelRate    

In [ ]:
#predictors = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

In [30]:
predictors = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']
scaler = StandardScaler().fit(diagnostics[predictors])
diagnostics_KNN = IterativeImputer().fit(scaler.transform(diagnostics[predictors]))

In [32]:
diagnostics_KNN.transform(scaler.transform(diagnostics[predictors]))

array([[-7.30208426e-01,  3.69721978e-03,  3.80149439e-01, ...,
         3.27774682e-01,  2.34983079e-01, -6.87081307e-01],
       [ 3.74160427e-02, -1.32443136e-03, -4.89655368e-02, ...,
        -1.11731388e-01,  3.16749148e-02, -9.79562286e-04],
       [ 3.74160427e-02, -1.32443136e-03, -4.89655368e-02, ...,
        -1.11731388e-01,  3.16749148e-02, -9.79562286e-04],
       ...,
       [-7.30208426e-01,  1.31139618e-01,  3.80149439e-01, ...,
         2.56640361e-02,  6.66238760e-01,  1.57228959e-01],
       [-6.85408077e-01,  1.94860817e-01,  4.58628704e-01, ...,
        -3.05023151e+00,  6.66238760e-01, -4.76003741e-01],
       [ 3.74160427e-02, -1.32443136e-03, -4.89655368e-02, ...,
        -1.11731388e-01,  3.16749148e-02, -9.79562286e-04]])

In [34]:
diagnostics[predictors] = diagnostics_KNN.transform(scaler.transform(diagnostics[predictors]))

In [35]:
diagnostics

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.730208,0.003697,False,0.380149,0.441701,-1.830304,-0.630095,-2.533521,-0.955860,-2.174952,-1.569446,-0.808218,-1.572512,-0.753686,-0.791132,False,-1.038278,1023,True,1.774131,-0.851694,0.327775,0.234983,-0.687081
2,0.037416,-0.001324,NaN,-0.048966,-0.000855,0.002983,-0.000027,-0.002967,-0.001909,-0.000089,0.004901,0.006814,-0.000890,-0.000379,-0.007855,True,-0.000865,1279,NaN,1.803153,-0.004423,-0.111731,0.031675,-0.000980
3,0.037416,-0.001324,NaN,-0.048966,-0.000855,0.002983,-0.000027,-0.002967,-0.001909,-0.000089,0.004901,0.006814,-0.000890,-0.000379,-0.007855,NaN,-0.000865,1279,NaN,1.803153,-0.004423,-0.111731,0.031675,-0.000980
4,0.037416,-0.001324,NaN,-0.048966,-0.000855,0.002983,-0.000027,-0.002967,-0.001909,-0.000089,0.004901,0.006814,-0.000890,-0.000379,-0.007855,True,-0.000865,1279,NaN,1.803153,-0.004423,-0.111731,0.031675,-0.000980
5,0.037416,-0.001324,NaN,-0.048966,-0.000855,0.002983,-0.000027,-0.002967,-0.001909,-0.000089,0.004901,0.006814,-0.000890,-0.000379,-0.007855,NaN,-0.000865,16639,NaN,1.803153,-0.004423,-0.111731,0.031675,-0.000980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,0.037416,-0.001324,NaN,-0.048966,-0.000855,0.002983,-0.000027,-0.002967,-0.001909,-0.000089,0.004901,0.006814,-0.000890,-0.000379,-0.007855,NaN,-0.000865,1023,NaN,1.803153,-0.004423,-0.111731,0.031675,-0.000980
1248455,2.069813,0.258582,True,0.262430,0.447058,0.516686,0.713102,0.388880,0.293957,0.876181,1.000833,1.539651,0.393082,0.615012,-0.218508,True,-0.284629,18431,False,0.147356,1.468689,-0.309293,0.066502,0.262768
1248456,-0.730208,0.131140,True,0.380149,0.743335,0.566622,1.082481,0.708518,0.308662,0.947205,0.606137,1.698530,0.649988,0.856130,-0.168048,True,-0.558683,17407,NaN,2.773793,1.524481,0.025664,0.666239,0.157229


In [ ]:
diagnostics

In [ ]:
diagnostics.dtypes

In [ ]:
diagnostics.isna().sum()

In [ ]:
diagnostics['ServiceDistance'].count()

In [ ]:
diagnostics.columns.values

In [ ]:
columns  = list(diagnostics.columns)

In [ ]:
predictors = ['']

**There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.**

In [5]:
faults = faults.drop(columns = ['ecuSoftwareVersion', 'ecuSerialNumber', 'faultValue', 'actionDescription', 'MCTNumber', 'ecuModel', 'ecuMake', 'ecuSource' ])

In [6]:
# Changing the data type of the latitude and longitude columns so that I can do a partial search 
# of the values that are near the service locations.
faults[['Latitude', 'Longitude']] = faults[['Latitude', 'Longitude']].astype(str)

In [14]:
station_1 = (faults['Latitude'].str.contains('36.06')) & faults['Longitude'].str.contains('86.43')
station_2 = (faults['Latitude'].str.contains('35.58')) & faults['Longitude'].str.contains('86.44')
station_3 = (faults['Latitude'].str.contains('36.19')) & faults['Longitude'].str.contains('83.17')
faults = faults.drop(faults[station_1 | station_2 | station_3].index)

In [9]:
# creating a column that has the lat long values together.
faults["lat_long"] = list(zip(faults['Latitude'], faults['Longitude']))

In [16]:
# convert dtypes as needed
# some columns have numeric-appearing values but they're not continuous variables, leaving them as strings
faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp'])
faults['activeTransitionCount'] = faults['activeTransitionCount'].astype(int)

In [17]:
# time of day for faults could be interesting, split event column to date and timestamp columns
faults['event_date'] = faults['EventTimeStamp'].dt.date
faults['event_time'] = faults['EventTimeStamp'].dt.time

In [21]:
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,"(38.857638, -84.626851)",2015-02-21,10:47:13
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,"(38.857638, -84.626851)",2015-02-21,11:34:34
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,11,1807,2,False,127,1369,105336226,41.42125,-87.767361,2015-02-21 11:35:26.000,"(41.42125, -87.767361)",2015-02-21,11:35:31
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,"(41.421018, -87.767361)",2015-02-21,11:35:33
4,5,990416,2015-02-21 11:39:41,NaN,NaN,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,"(38.416481, -89.442638)",2015-02-21,11:39:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,6X1u17D1500000000,CMMNS,0,1761,17,False,3,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,"(37.094768, -85.897407)",2020-03-06,14:00:26
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.39074,-79.461805,2020-03-06 14:04:59.000,"(34.39074, -79.461805)",2020-03-06,14:04:23
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.43037,-84.920509,2020-03-06 14:14:14.000,"(34.43037, -84.920509)",2020-03-06,14:13:38
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,"(35.030925, -85.321527)",2020-03-06,14:14:13


In [ ]:
#faults[(faults['Latitude'].str.contains(r'36.066|35.58|36.19')) & (faults['Longitude'].str.contains(r'-86.43|-86.44|-83.17'))]

In [ ]:
#faults[faults["lat_long"].str.contains(r'36.066'and'-86.43')]

In [ ]:
#faults[faults["lat_long"].str.contains(r'36.066') & faults["lat_long"].str.contains(r'-86.43')]


In [ ]:
# removing the rows that had Lat and long values near the service locations.
#faults = faults.drop(faults[faults['Latitude'].str.contains(r'36.066|35.58|36.19')].index)

Break down Location Time stamp, into days and hours?

In [ ]:
pd.to_datetime(faults['LocationTimeStamp],
                format = "%Y/%m/%d")